In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os
import IPython.display as display
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

print(tf.__version__)
print(keras.__version__)

2.0.0
2.2.4-tf


In [2]:
tf.executing_eagerly()

True

In [3]:
tfrecord_train = 'train.tfrecords'
tfrecord_test = 'test.tfrecords'
tfrecord_dir = 'tfrecords'

n_train = 21172
n_test = 5293

cur_dir = os.getcwd()
train_tfr_path = os.path.join(cur_dir, tfrecord_dir, tfrecord_train)
print(train_tfr_path)
test_tfr_path = os.path.join(cur_dir, tfrecord_dir, tfrecord_test)

C:\Users\beplu\Desktop\ForJupyter\blurryDetector\tfrecords\train.tfrecords


In [4]:
batch_size = 64
epochs = 15
IMG_HEIGHT = 64
IMG_WIDTH = 64

In [5]:
def _parsed_function(tfrecord_serialized):
    features={'image_raw': tf.io.FixedLenFeature([], tf.string),
             'label': tf.io.FixedLenFeature([], tf.int64),
             'height': tf.io.FixedLenFeature([], tf.int64),
            'width': tf.io.FixedLenFeature([], tf.int64),
            'depth': tf.io.FixedLenFeature([], tf.int64),}
    return tf.io.parse_single_example(tfrecord_serialized, features)

In [6]:
def decode_image_function(dataset):
    print(dataset['image_raw'])
    dataset['image_raw'] = tf.image.decode_image(dataset['image_raw'])
    dataset['image_raw'] = tf.image.convert_image_dtype(dataset['image_raw'], tf.float32)
    print(dataset['image_raw'])
    #print(image.numpy().shape)
    #image = tf.cast(tf.image.decode_image(dataset['image_raw']), tf.float32)/255.
    #image = tf.image.resize(image, size=(IMG_HEIGHT,IMG_WIDTH))
    #print(image)
    dataset['label'] = tf.cast(dataset['label'], tf.int32)
    #label = tf.one_hot(label, depth=n_class)
    print('label = ', dataset['label'])
    return dataset

In [7]:
train_dataset = tf.data.TFRecordDataset(train_tfr_path)
train_dataset = train_dataset.map(_parsed_function)
train_dataset = train_dataset.map(decode_image_function)

print('dataset = ', train_dataset)
#train_dataset = train_dataset.map(_parse_function, num_parallel_calls=8)
train_dataset = train_dataset.shuffle(buffer_size=n_train*2).batch(batch_size)
#print(train_dataset)
test_dataset = tf.data.TFRecordDataset(test_tfr_path)
test_dataset = test_dataset.map(_parsed_function)
test_dataset = test_dataset.map(decode_image_function)
#test_dataset = test_dataset.map(_parse_function, num_parallel_calls=8)
test_dataset = test_dataset.shuffle(buffer_size=n_test*2).batch(batch_size)

Tensor("args_2:0", shape=(), dtype=string)
Tensor("convert_image:0", dtype=float32)
label =  Tensor("Cast:0", shape=(), dtype=int32)
dataset =  <MapDataset shapes: {depth: (), height: (), image_raw: <unknown>, label: (), width: ()}, types: {depth: tf.int64, height: tf.int64, image_raw: tf.float32, label: tf.int32, width: tf.int64}>
Tensor("args_2:0", shape=(), dtype=string)
Tensor("convert_image:0", dtype=float32)
label =  Tensor("Cast:0", shape=(), dtype=int32)


In [8]:
model = Sequential([
    Conv2D(32, 3, activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(),
    Conv2D(32, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(2, activation='softmax')
])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0

In [10]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.fit(train_dataset, epochs=10, verbose=2)

Epoch 1/10


ValueError: as_list() is not defined on an unknown TensorShape.

In [ ]:
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu, padding='SAME', input_shape=(250, 250, 3)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(2))
    return model